In [2]:
import PIL
from PIL import Image 
from PIL import ImageFont
from PIL import ImageDraw
from PIL import ImageChops
from PIL import ImageFilter

# Get Cyan RGB values

In [2]:
# open cyan.png
cyan = Image.open("cyan.png")

# get rgb values of cyan.png
cyan = cyan.convert("RGB")
# print the rgb values of cyan.png
print(cyan.getpixel((0, 0)))

(83, 234, 205)


# Get dark grey rgb values


In [15]:
# open dark_grey.png
dark_grey = Image.open("pixegami_dark_grey.png")

# get rgb values of dark_grey.png
dark_grey = dark_grey.convert("RGB")

# print the rgb values of dark_grey.png
print(dark_grey.getpixel((0, 0)))

(30, 41, 60)


# Get dark blue (howai) rgb values

In [63]:
dark_blue = Image.open("dark_blue.png")

# get rgb values of dark_blue.png
dark_blue = dark_blue.convert("RGB")

# print the rgb values of dark_blue.png
print(dark_blue.getpixel((0, 0)))

(52, 56, 130)


# get monokai bg color

In [162]:
bg_monokai = Image.open("bg_monokai.png")

# get rgb values of bg_monokai.png
bg_monokai = bg_monokai.convert("RGB")

# print the rgb values of bg_monokai.png
print(bg_monokai.getpixel((0, 0)))


(43, 44, 39)


# get lightbulb bg color

In [163]:
bg_lightbulb = Image.open("bg_lightbulb.png")

# get rgb values of bg_lightbulb.png
bg_lightbulb = bg_lightbulb.convert("RGB")

# print the rgb values of bg_lightbulb.png
print(bg_lightbulb.getpixel((0, 0)))

(34, 40, 52)


# Create image with grey box in the middle

In [2]:
# create a new image and put white pixels in it using rgb values of white
white = PIL.Image.new("RGB", (800, 500), "white")

# write text on the image
font = ImageFont.truetype("arialbd.ttf", 50)
draw = PIL.ImageDraw.Draw(white)

# get the size of the text
bbox = draw.textbbox((0, 0), "HI", font=font)
text_size = (bbox[2] - bbox[0], bbox[3] - bbox[1])
draw.text(((white.size[0]-text_size[0])/2, (white.size[1]-text_size[1])/2), "HI", font=font, fill="black")

# Create a new image with a grey background
gradient = PIL.Image.new("RGB", (text_size[0], text_size[1]+100), "grey")

for y in range(gradient.size[1]):
    for x in range(gradient.size[0]):
        gradient.putpixel((x, y), (255-int(172*x/gradient.size[0]), 255-int(21*x/gradient.size[0]), 255-int(50*x/gradient.size[0])))

# Place the new image on the white image
white.paste(gradient, (int((white.size[0]-gradient.size[0])/2), int((white.size[1]-gradient.size[1])/2)))
white.show()

# put text in the middle

In [87]:

def create_text_cutout_image(width=800, height=600, text="Hello World", font_size=150):
    # Step 1: Create a white background
    background = Image.new("RGB", (width, height), (30, 41, 60))#"grey")

    # create a new image and put white pixels in it using rgb values of white
    white = PIL.Image.new("RGB", (width, height), "white")

    # write text on the image
    font = ImageFont.truetype("arialbd.ttf", font_size)
    draw = PIL.ImageDraw.Draw(white)

    # Calculate text position to center it
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
    text_x = (width - text_width) // 2
    text_y = (height - text_height) // 2


    # Create a new image with a grey background
    gradient = PIL.Image.new("RGB", (text_width, text_height+100), "grey")

    # Create Gradient from white to cyan
    for y in range(gradient.size[1]):
        for x in range(gradient.size[0]):
            gradient.putpixel((x, y), (255-int(172*x/gradient.size[0]), 255-int(21*x/gradient.size[0]), 255-int(50*x/gradient.size[0])))

    # Place the new image on the white image
    white.paste(gradient, (int((white.size[0]-gradient.size[0])/2), int((white.size[1]-gradient.size[1])/2)))

    # Step 3: Create a mask for the text
    mask = Image.new("L", (width, height), 0)  # Black (0) background for mask
    draw = ImageDraw.Draw(mask)
    
    # Draw white text on the black mask
    draw.text((text_x, text_y), text, fill=255, font=font)  # White text on black mask

    # Step 4: Apply the mask to create transparency in the gray image
    white.putalpha(mask)

    # Create piece of canvas to draw text on and blur
    blurred = Image.new('RGBA', background.size)
    draw = ImageDraw.Draw(blurred)
    draw.text(xy=(text_x,text_y), text=text, fill='black', font=font)#, anchor='mm')
    blurred = blurred.filter(ImageFilter.BoxBlur(50))

    # Create an alpha mask for the blurred image with 70% opacity
    alpha = blurred.split()[3]  # Get the alpha channel
    alpha = alpha.point(lambda p: int(p * 0.7))  # Adjust opacity to 70%
    blurred.putalpha(alpha)  # Apply the new alpha channel to the blurred image

    # Paste soft text onto background
    background.paste(blurred,blurred)

    # Step 5: Composite the images together (white background, gray with transparent text)
    result = Image.alpha_composite(background.convert("RGBA"), white)

    return result

# Usage
image = create_text_cutout_image(1920, 1080, "howai.works")
image.show()

# save the image
image.save("howai_works.png")


In [90]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import PIL

class TextCutoutImage:
    """
    A class to create stylized text cutout images with gradient effects and blurring.
    
    The image consists of a background with a gradient text overlay and blur effects.
    """
    
    def __init__(self, width=800, height=600, background_color=(30, 41, 60), vertical_offset=100, background="gradient canvas.png"):
        """
        Initialize the TextCutoutImage with basic parameters.
        
        Args:
            width (int): Width of the image in pixels
            height (int): Height of the image in pixels
            background_color (tuple): RGB color tuple for the background
            vertical_offset (int): Number of pixels to shift text upward from center
        """
        self.width = width
        self.height = height
        self.background_color = background_color
        self.vertical_offset = vertical_offset
        self.gradient_canvas = Image.open(background)
        
    def _create_gradient(self, width, height):
        """
        Create a gradient image transitioning from white to a custom color.
        
        Args:
            width (int): Width of the gradient
            height (int): Height of the gradient
        
        Returns:
            PIL.Image: The generated gradient image
        """
        gradient = Image.new("RGB", (width, height+200), "grey")
        for y in range(gradient.size[1]):
            for x in range(gradient.size[0]):
                # Create gradient from white to custom color
                r = 255 - int(172 * x / gradient.size[0])
                g = 255 - int(21 * x / gradient.size[0])
                b = 255 - int(50 * x / gradient.size[0])
                gradient.putpixel((x, y), (r, g, b))
        return gradient
    
    def _calculate_text_position(self, text, font):
        """
        Calculate the position to center the text horizontally and shift it upward.
        
        Args:
            text (str): The text to be centered
            font (ImageFont): The font object to use
        
        Returns:
            tuple: (x, y) coordinates for text placement and text dimensions
        """
        draw = ImageDraw.Draw(Image.new("RGB", (1, 1)))
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        text_x = (self.width - text_width) // 2
        # Calculate vertical center and then shift upward by vertical_offset
        text_y = ((self.height - text_height) // 2) - self.vertical_offset
        return text_x, text_y, text_width, text_height
    
    def _create_blurred_text(self, text, font, text_x, text_y):
        """
        Create a blurred version of the text with custom opacity.
        
        Args:
            text (str): The text to blur
            font (ImageFont): The font object to use
            text_x (int): X coordinate for text placement
            text_y (int): Y coordinate for text placement
            
        Returns:
            PIL.Image: Blurred text image with alpha channel
        """
        blurred = Image.new('RGBA', (self.width, self.height))
        draw = ImageDraw.Draw(blurred)
        draw.text((text_x, text_y), text, fill='black', font=font)
        blurred = blurred.filter(ImageFilter.BoxBlur(50))
        
        # Adjust opacity to 70%
        #alpha = blurred.split()[3]
        #alpha = alpha.point(lambda p: int(p * 0.7))
        #blurred.putalpha(alpha)
        return blurred
    
    def create_image(self, text, font_path="arialbd.ttf", font_size=150):
        """
        Create the final text cutout image with all effects applied.
        """
        # Create background
        background = self.gradient_canvas
        
        # Create two white canvases - one for sharp text, one for blurred
        white_sharp = Image.new("RGB", (self.width, self.height), "white")
        white_blur = Image.new("RGB", (self.width, self.height), "white")
        
        # Set up font
        font = ImageFont.truetype(font_path, font_size)
        
        # Calculate text position
        text_x, text_y, text_width, text_height = self._calculate_text_position(text, font)
        
        # Create and apply gradient
        gradient = self._create_gradient(text_width, text_height)
        
        # Apply gradient to both canvases
        gradient_pos = (
            int((white_sharp.size[0] - gradient.size[0]) / 2),
            int((white_sharp.size[1] - gradient.size[1]) / 2) - self.vertical_offset
        )
        white_sharp.paste(gradient, gradient_pos)
        white_blur.paste(gradient, gradient_pos)
        
        # Create masks for both sharp and blur versions
        mask_sharp = Image.new("L", (self.width, self.height), 0)
        mask_blur = Image.new("L", (self.width, self.height), 0)
        
        draw_sharp = ImageDraw.Draw(mask_sharp)
        draw_blur = ImageDraw.Draw(mask_blur)
        
        # Draw text on both masks
        draw_sharp.text((text_x, text_y), text, fill=255, font=font)
        draw_blur.text((text_x, text_y), text, fill=180, font=font)
        
        # Apply the sharp mask
        white_sharp.putalpha(mask_sharp)
        
        # Create blurred version
        # First apply the mask to get gradient text
        white_blur.putalpha(mask_blur)
        # Then apply blur to the entire gradient text
        white_blur = white_blur.filter(ImageFilter.GaussianBlur(20))
        
        # Adjust blur opacity if needed
        if isinstance(white_blur, PIL.Image.Image):
            white_blur = white_blur.convert('RGBA')
        blur_bands = list(white_blur.split())
        if len(blur_bands) == 4:
            blur_bands[3] = blur_bands[3].point(lambda x: int(x * 0.7))
            white_blur = Image.merge('RGBA', blur_bands)
        
        # Composite all layers
        background = background.convert("RGBA")
        # First add the blur/glow
        result = Image.alpha_composite(background, white_blur)
        # Then add the sharp text
        result = Image.alpha_composite(result, white_sharp)
        
        return result


creator = TextCutoutImage(3840, 2160, vertical_offset=120)  # Added vertical offset
image = creator.create_image("howai.works", font_size=300)
image.show()

# radial blur

In [121]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import PIL

class TextCutoutImage:
    """
    A class to create stylized text cutout images with gradient effects and blurring.
    
    The image consists of a background with a gradient text overlay and blur effects.
    """
    
    def __init__(self, width=800, height=600, background_color=(30, 41, 60), vertical_offset=100, background="brain_on_gradient_canvas.png"):
        """
        Initialize the TextCutoutImage with basic parameters.
        
        Args:
            width (int): Width of the image in pixels
            height (int): Height of the image in pixels
            background_color (tuple): RGB color tuple for the background
            vertical_offset (int): Number of pixels to shift text upward from center
        """
        self.width = width
        self.height = height
        self.background_color = background_color
        self.vertical_offset = vertical_offset
        self.gradient_canvas = Image.open(background)
        
    def _create_gradient(self, width, height):
        """
        Create a circular gradient image transitioning from white at center to specified color at edges.
        
        Args:
            width (int): Width of the gradient
            height (int): Height of the gradient
        
        Returns:
            PIL.Image: The generated gradient image
        """
        gradient = Image.new("RGB", (width, height+200), "white")
        draw = ImageDraw.Draw(gradient)
        
        # Center coordinates
        center_x = width // 2
        center_y = (height + 200) // 2
        
        # Maximum distance from center to corner (this will be our radius)
        max_radius = int(((width/2)**2 + ((height+200)/2)**2)**0.5)
        
        # Target RGB values
        target_r, target_g, target_b = (83, 234, 205)
        
        # Create gradient pixel by pixel
        for y in range(gradient.size[1]):
            for x in range(gradient.size[0]):
                # Calculate distance from center
                distance = ((x - center_x)**2 + (y - center_y)**2)**0.5
                
                # Create normalized ratio (0 at center, 1 at max_radius)
                ratio = min(distance / max_radius, 1.0)
                
                # Linear interpolation between white (255, 255, 255) and target color
                r = int(255 - (255 - target_r) * ratio)
                g = int(255 - (255 - target_g) * ratio)
                b = int(255 - (255 - target_b) * ratio)
                
                gradient.putpixel((x, y), (r, g, b))
                
        return gradient
    
    def _calculate_text_position(self, text, font):
        """
        Calculate the position to center the text horizontally and shift it upward.
        
        Args:
            text (str): The text to be centered
            font (ImageFont): The font object to use
        
        Returns:
            tuple: (x, y) coordinates for text placement and text dimensions
        """
        draw = ImageDraw.Draw(Image.new("RGB", (1, 1)))
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        text_x = (self.width - text_width) // 2
        # Calculate vertical center and then shift upward by vertical_offset
        text_y = ((self.height - text_height) // 2) - self.vertical_offset
        return text_x, text_y, text_width, text_height
    
    def _create_blurred_text(self, text, font, text_x, text_y):
        """
        Create a blurred version of the text with custom opacity.
        
        Args:
            text (str): The text to blur
            font (ImageFont): The font object to use
            text_x (int): X coordinate for text placement
            text_y (int): Y coordinate for text placement
            
        Returns:
            PIL.Image: Blurred text image with alpha channel
        """
        blurred = Image.new('RGBA', (self.width, self.height))
        draw = ImageDraw.Draw(blurred)
        draw.text((text_x, text_y), text, fill='black', font=font)
        blurred = blurred.filter(ImageFilter.BoxBlur(50))
        
        # Adjust opacity to 70%
        #alpha = blurred.split()[3]
        #alpha = alpha.point(lambda p: int(p * 0.7))
        #blurred.putalpha(alpha)
        return blurred
    
    def create_image(self, text, font_path="arialbd.ttf", font_size=150):
        """
        Create the final text cutout image with all effects applied.
        """
        # Create background
        background = self.gradient_canvas
        
        # Create two white canvases - one for sharp text, one for blurred
        white_sharp = Image.new("RGB", (self.width, self.height), "white")
        white_blur = Image.new("RGB", (self.width, self.height), "white")
        
        # Set up font
        font = ImageFont.truetype(font_path, font_size)
        
        # Calculate text position
        text_x, text_y, text_width, text_height = self._calculate_text_position(text, font)
        
        # Create and apply gradient
        gradient = self._create_gradient(text_width, text_height)
        
        # Apply gradient to both canvases
        gradient_pos = (
            int((white_sharp.size[0] - gradient.size[0]) / 2),
            int((white_sharp.size[1] - gradient.size[1]) / 2) - self.vertical_offset
        )
        white_sharp.paste(gradient, gradient_pos)
        white_blur.paste(gradient, gradient_pos)
        
        # Create masks for both sharp and blur versions
        mask_sharp = Image.new("L", (self.width, self.height), 0)
        mask_blur = Image.new("L", (self.width, self.height), 0)
        
        draw_sharp = ImageDraw.Draw(mask_sharp)
        draw_blur = ImageDraw.Draw(mask_blur)
        
        # Draw text on both masks
        draw_sharp.text((text_x, text_y), text, fill=255, font=font)
        draw_blur.text((text_x, text_y), text, fill=180, font=font)
        
        # Apply the sharp mask
        white_sharp.putalpha(mask_sharp)
        
        # Create blurred version
        # First apply the mask to get gradient text
        white_blur.putalpha(mask_blur)
        # Then apply blur to the entire gradient text
        white_blur = white_blur.filter(ImageFilter.GaussianBlur(20))
        
        # Adjust blur opacity if needed
        if isinstance(white_blur, PIL.Image.Image):
            white_blur = white_blur.convert('RGBA')
        blur_bands = list(white_blur.split())
        if len(blur_bands) == 4:
            blur_bands[3] = blur_bands[3].point(lambda x: int(x * 0.7))
            white_blur = Image.merge('RGBA', blur_bands)
        
        # Composite all layers
        background = background.convert("RGBA")
        # First add the blur/glow
        result = Image.alpha_composite(background, white_blur)
        # Then add the sharp text
        result = Image.alpha_composite(result, white_sharp)
        
        return result


creator = TextCutoutImage(3840, 2160, vertical_offset=120)  # Added vertical offset
image = creator.create_image("Daniel", font_size=300)
image.show()
image.save("Daniel.png")

# blurred rectangle for seperation

In [118]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import PIL

class TextCutoutImage:
    """
    A class to create stylized text cutout images with gradient effects and blurring.
    
    The image consists of a background with a gradient text overlay and blur effects.
    """
    
    def __init__(self, width=800, height=600, background_color=(30, 41, 60), vertical_offset=100, background="brain_on_gradient_canvas.png"):
        """
        Initialize the TextCutoutImage with basic parameters.
        
        Args:
            width (int): Width of the image in pixels
            height (int): Height of the image in pixels
            background_color (tuple): RGB color tuple for the background
            vertical_offset (int): Number of pixels to shift text upward from center
        """
        self.width = width
        self.height = height
        self.background_color = background_color
        self.vertical_offset = vertical_offset
        self.gradient_canvas = Image.open(background)
        
    def _create_gradient(self, width, height, center=None):
        """
        Create a circular gradient image transitioning from white at center to specified color at edges.
        """
        gradient = Image.new("RGB", (width, height), "white")
        draw = ImageDraw.Draw(gradient)
        
        # Use provided center or calculate center
        center_x = center[0] if center else width // 2
        center_y = center[1] if center else height // 2
        
        # Maximum distance from center to corner
        max_radius = int(((width/2)**2 + (height/2)**2)**0.5)
        
        # Target RGB values
        target_r, target_g, target_b = (83, 234, 205)
        
        # Create gradient pixel by pixel
        for y in range(height):
            for x in range(width):
                # Calculate distance from center
                distance = ((x - center_x)**2 + (y - center_y)**2)**0.5
                
                # Create normalized ratio (0 at center, 1 at max_radius)
                ratio = min(distance / max_radius, 1.0)
                
                # Linear interpolation between white (255, 255, 255) and target color
                r = int(255 - (255 - target_r) * ratio)
                g = int(255 - (255 - target_g) * ratio)
                b = int(255 - (255 - target_b) * ratio)
                
                gradient.putpixel((x, y), (r, g, b))
        
        return gradient
        
  
    def _calculate_text_position(self, text, font):
        """
        Calculate the position to center the text horizontally and shift it upward.
        
        Args:
            text (str): The text to be centered
            font (ImageFont): The font object to use
        
        Returns:
            tuple: (x, y) coordinates for text placement and text dimensions
        """
        draw = ImageDraw.Draw(Image.new("RGB", (1, 1)))
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]
        text_x = (self.width - text_width) // 2
        # Calculate vertical center and then shift upward by vertical_offset
        text_y = ((self.height - text_height) // 2) - self.vertical_offset
        return text_x, text_y, text_width, text_height
    
    def _create_blurred_text(self, text, font, text_x, text_y):
        """
        Create a blurred version of the text with custom opacity.
        
        Args:
            text (str): The text to blur
            font (ImageFont): The font object to use
            text_x (int): X coordinate for text placement
            text_y (int): Y coordinate for text placement
            
        Returns:
            PIL.Image: Blurred text image with alpha channel
        """
        blurred = Image.new('RGBA', (self.width, self.height))
        draw = ImageDraw.Draw(blurred)
        draw.text((text_x, text_y), text, fill='black', font=font)
        blurred = blurred.filter(ImageFilter.BoxBlur(50))
        
        # Adjust opacity to 70%
        #alpha = blurred.split()[3]
        #alpha = alpha.point(lambda p: int(p * 0.7))
        #blurred.putalpha(alpha)
        return blurred
    
    def create_image(self, text, font_path="arialbd.ttf", font_size=150):
        """
        Create the final text cutout image with all effects applied.
        """
        # Create background
        background = self.gradient_canvas
        white_sharp = Image.new("RGB", (self.width, self.height), "white")
        white_blur = Image.new("RGB", (self.width, self.height), "white")
        
        # Set up font
        font = ImageFont.truetype(font_path, font_size)
        
        # Calculate text position and size
        text_x, text_y, text_width, text_height = self._calculate_text_position(text, font)
        
        # Calculate the true center of the text
        text_center_x = text_x + (text_width // 2)
        text_center_y = text_y + (text_height // 2)
        
        # Create gradient first, centered on text
        padding = 100
        total_width = text_width + (2 * padding)
        total_height = text_height + (2 * padding)
        
        gradient = self._create_gradient(total_width, total_height, (text_width // 2, text_height // 2))
        
        # Calculate where to paste the gradient so it centers on the text
        gradient_x = text_center_x - (total_width // 2)
        gradient_y = text_center_y - (total_height // 2)
        
        white_sharp.paste(gradient, (gradient_x, gradient_y))
        white_blur.paste(gradient, (gradient_x, gradient_y))
        
        # Create background box
        box = Image.new('RGBA', (self.width, self.height), (255, 255, 255, 0))
        draw_box = ImageDraw.Draw(box)
        
        # Draw a rectangle centered on text
        box_coords = (
            text_center_x - (text_width // 2) - padding,
            text_center_y - (text_height // 2) - padding,
            text_center_x + (text_width // 2) + padding,
            text_center_y + (text_height // 2) + padding
        )
        draw_box.rectangle(box_coords, fill=(50, 50, 50, 40))
        
        # Blur the box
        box = box.filter(ImageFilter.GaussianBlur(30))
        
        # Create masks for both sharp and blur versions
        mask_sharp = Image.new("L", (self.width, self.height), 0)
        mask_blur = Image.new("L", (self.width, self.height), 0)
        
        draw_sharp = ImageDraw.Draw(mask_sharp)
        draw_blur = ImageDraw.Draw(mask_blur)
        
        # Draw text on both masks
        draw_sharp.text((text_x, text_y), text, fill=255, font=font)
        draw_blur.text((text_x, text_y), text, fill=255, font=font)
        
        # Apply the sharp mask
        white_sharp.putalpha(mask_sharp)
        
        # Create blurred version
        white_blur.putalpha(mask_blur)
        white_blur = white_blur.filter(ImageFilter.GaussianBlur(30))
        
        # Adjust blur opacity
        if isinstance(white_blur, PIL.Image.Image):
            white_blur = white_blur.convert('RGBA')
        blur_bands = list(white_blur.split())
        if len(blur_bands) == 4:
            blur_bands[3] = blur_bands[3].point(lambda x: int(x * 0.7))
            white_blur = Image.merge('RGBA', blur_bands)
        
        # Composite all layers
        background = background.convert("RGBA")
        result = Image.alpha_composite(background, box)
        result = Image.alpha_composite(result, white_blur)
        result = Image.alpha_composite(result, white_sharp)
        
        return result


creator = TextCutoutImage(3840, 2160, vertical_offset=120)  # Added vertical offset
image = creator.create_image("Daniel", font_size=300)
image.show()
image.save("Daniel.png")

# brain on white canvas

In [100]:
from PIL import Image

# Load the original image
original_image = Image.open("brain.png")


# Define the canvas size (adjust as needed, here it is 500x500)
canvas_width = 500
canvas_height = 500

# Create a white canvas
canvas =  Image.open("gradient canvas32.png")#Image.new("RGB", (canvas_width, canvas_height), "white")

# Convert the original image to RGBA (if not already) to handle transparency
if original_image.mode != 'RGBA':
    original_image = original_image.convert('RGBA')

# Create a white background image of the same size
white_bg = Image.new("RGBA", original_image.size, (25, 25, 25, 25))

# Composite the original image on top of the white background
composite_image = Image.alpha_composite(white_bg, original_image)

# Convert the composite image back to RGB
composite_image = composite_image.convert("RGB")

# Calculate the position to center the original image on the canvas
x_offset = (canvas_width - composite_image.width) // 2
y_offset = (canvas_height - composite_image.height) // 2

#composite_image.show()

# Paste the composite image onto the white canvas
canvas.paste(composite_image, (x_offset, y_offset))

# Save or display the result
canvas.show()  # Use this to display the image


# brain on gradient canvas

In [50]:
from PIL import Image, ImageOps

# Load the gradient canvas
gradient_canvas = Image.open("gradient canvas.png")

# Load the original brain image
brain_image = Image.open("brain_1500.png")

# upscale the brain image
#brain_image = brain_image.resize((int(brain_image.width*1.4), int(brain_image.height*1.4)))

# Convert the brain icon to RGBA to handle transparency
if brain_image.mode != 'RGBA':
    brain_image = brain_image.convert('RGBA')

# Change the color of the brain icon to (100, 100, 100)
# This involves replacing each non-transparent pixel with the specified color
#r, g, b, a = brain_image.split()  # Split into individual channels
#gray_brain = Image.merge("RGBA", (r.point(lambda _: 100), g.point(lambda _: 100), b.point(lambda _: 100), a))
gray_brain = brain_image

# Calculate the position to center the brain image on the gradient canvas
x_offset = (gradient_canvas.width - gray_brain.width) // 2
y_offset = (gradient_canvas.height - gray_brain.height) // 2

# Create blurred version of the brain icon
blurred_brain = gray_brain.filter(ImageFilter.GaussianBlur(50))

# Composite the brain icon onto the gradient canvas
gradient_canvas.paste(blurred_brain, (x_offset, y_offset), blurred_brain)
gradient_canvas.paste(gray_brain, (x_offset, y_offset), gray_brain)

# Save or display the result
gradient_canvas.show()  # Use this to display the image
#gradient_canvas.save("brain_on_gradient_canvas.png")  # Save the result


In [101]:
from PIL import Image

# Load the gradient canvas
gradient_canvas = Image.open("gradient canvas.png")

# Load the original brain image
brain_image = Image.open("brain_1500.png")

# Ensure the brain image has an alpha channel
if brain_image.mode != 'RGBA':
    brain_image = brain_image.convert('RGBA')

# Create a new image to apply the color
colored_brain = Image.new("RGBA", brain_image.size)

# Apply the target color (52, 56, 130) to each non-transparent pixel
for x in range(brain_image.width):
    for y in range(brain_image.height):
        r, g, b, a = brain_image.getpixel((x, y))
        if a > 0:  # If the pixel is not fully transparent
            colored_brain.putpixel((x, y), (52, 56, 130, a))
        else:
            colored_brain.putpixel((x, y), (0, 0, 0, 0))  # Keep transparent pixels transparent

# Calculate the position to center the brain image on the gradient canvas
x_offset = (gradient_canvas.width - colored_brain.width) // 2
y_offset = (gradient_canvas.height - colored_brain.height) // 2

# Composite the colored brain icon onto the gradient canvas
gradient_canvas.paste(colored_brain, (x_offset, y_offset), colored_brain)

# Save or display the result
gradient_canvas.show()  # Use this to display the image
gradient_canvas.save("brain_on_gradient_canvas.png")  # Save the result


In [27]:
# import pygments
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import ImageFormatter
import io

# Define the Python code to highlight
code = """
def hello_world():
    print("Hello, World!")
    return TRUE
"""

# Write the code onto an image with a rounded rectangle background from the pillow library
image = Image.new("RGB", (800, 600), "white")
draw = ImageDraw.Draw(image)
draw.rounded_rectangle((50, 50, 750, 550), radius=10, outline="black", fill=(35, 35, 35), width=5)

# Create a BytesIO buffer to store the image data
buffer = io.BytesIO()

# Use Pygments to highlight the code and format it as an image
highlight(code, PythonLexer(), ImageFormatter(font_name="arial", font_size=20, line_numbers=False), outfile=buffer)

# Load the image data from the buffer
buffer.seek(0)
highlighted_code = Image.open(buffer)

# Paste the highlighted code onto the image
image.paste(highlighted_code, (60, 60))

# Save or display the image
image.show()  # Use this to display the image
#image.save("highlighted_code.png")  # Save the image

In [123]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import ImageFormatter
from PIL import Image, ImageDraw, ImageOps
import io
from pygments.styles import get_style_by_name


# Define the Python code to highlight
code = """
def hello_world():
    print("Hello, World!")
    return TRUE
"""

# Create the base image with a rounded rectangle background
image = Image.open("cyan blue.png")
draw = ImageDraw.Draw(image)

# Create a BytesIO buffer to store the image data
buffer = io.BytesIO()

# Use Pygments to highlight the code without line numbers and format it as an image
monokai_style = get_style_by_name("lightbulb")
formatter = ImageFormatter(font_name='Consolas', font_size=50, line_numbers=False, style=monokai_style, line_pad=15)
highlight(code, PythonLexer(), formatter, outfile=buffer)

# Load the image data from the buffer
buffer.seek(0)
highlighted_code = Image.open(buffer)

# get the size of the highlighted code
highlighted_code_width, highlighted_code_height = highlighted_code.size
print(highlighted_code.size)


# Paste the highlighted code with background onto the image
image.paste(highlighted_code, (int(image.width // 2.5), int(image.height // 2.3)))

# Save or display the image
image.show()  # Use this to display the image


(722, 179)


In [76]:
from pygments import highlight
from pygments.formatters import ImageFormatter
from pygments.lexers import PythonLexer

# create a white image
image = Image.new("RGB", (1200, 600), "black")

code = '''
def hello_world():
    print("Hello, World!")
    return True
'''

formatter = ImageFormatter(
    font_size=50,         # Base font size
    line_numbers=False,    # Optional: include line numbers
    style='lightbulb',      # Choose your preferred style
    line_pad=10           # Adjust this value to change spacing between lines
)

# Generate the image
with open('output.png', 'wb') as f:
    highlight(code, PythonLexer(), formatter, f)

image.paste(Image.open("output.png"), (100, 100))
image.show()



# claude for code snippets

In [65]:
from PIL import Image, ImageDraw, ImageFilter
from pygments import highlight
from pygments.lexers import PythonLexer, get_lexer_by_name
from pygments.formatters import ImageFormatter
from pygments.styles import get_style_by_name
import io

class CodeSnippetVisualizer:
    def __init__(self, background_image_path, 
                 font_name='Consolas', 
                 font_size=80,
                 font_style='monokai',
                 min_width=2500,  # Added minimum width parameter
                 padding_left=100,  # Separate padding for each side
                 padding_right=100,
                 padding_top=125,#175,    # More padding on top
                 padding_bottom=30,#80,  # Less padding on bottom
                 background_color=(34, 40, 52),#(35, 35, 35),
                 border_color=(83, 234, 205),
                 border_width=5,
                 corner_radius=20,
                 language='python'):
        self.background_image = Image.open(background_image_path)
        self.font_name = font_name
        self.font_size = font_size
        self.font_style = font_style
        self.min_width = min_width
        self.padding = (padding_left, padding_right, padding_top, padding_bottom)
        self.background_color = background_color
        self.border_color = (*border_color, 255)
        self.border_width = border_width
        self.corner_radius = corner_radius
        self.language = language

    def _create_code_image(self, code, language='python'):
        """Create an image of the highlighted code."""
        buffer = io.BytesIO()
        
        lexer = get_lexer_by_name(language)
        style = get_style_by_name(self.font_style)
        
        formatter = ImageFormatter(
            font_name=self.font_name,
            font_size=self.font_size,
            line_numbers=False,
            style=style,
            line_pad=30,
            image_pad=0,
            background_color='transparent'
        )
        
        highlight(code, lexer, formatter, outfile=buffer)
        buffer.seek(0)
        
        code_image = Image.open(buffer).convert('RGBA')
        return code_image

    def _draw_control_buttons(self, draw, x0, y0):
        """Draw the three control buttons (red, yellow, green)."""
        button_positions = [
            ((255, 96, 92), 115),
            ((255, 189, 68), 200),
            ((0, 202, 78), 285)
        ]
        
        for color, x_offset in button_positions:
            draw.circle((x0 + x_offset, y0 + 100), 28, fill=color)

    def create_visualization(self, code, language='python'):
        # Create the code image first to get its dimensions
        code_image = self._create_code_image(code, language)
        code_width, code_height = code_image.size
        
        # Calculate rectangle dimensions with padding
        padding_left, padding_right, padding_top, padding_bottom = self.padding
        rect_width = max(self.min_width, code_width + padding_left + padding_right)
        rect_height = code_height + padding_top + padding_bottom
        
        # Calculate rectangle position (centered horizontally)
        center_x = self.background_image.width // 2
        center_y = self.background_image.height // 2
        x0 = center_x - rect_width // 2
        y0 = center_y - rect_height // 2
        x1 = x0 + rect_width
        y1 = y0 + rect_height
        
        # Create a copy of the background image
        result = self.background_image.copy().convert('RGBA')
        
        # Create and apply the blurred background
        blurred = Image.new('RGBA', result.size, (0, 0, 0, 0))
        draw_blurred = ImageDraw.Draw(blurred)
        draw_blurred.rounded_rectangle(
            (x0, y0, x1, y1),
            radius=self.corner_radius,
            fill=(*self.border_color[:3], 255)
        )
        blurred = blurred.filter(ImageFilter.GaussianBlur(50))
        result = Image.alpha_composite(result, blurred)
        
        # Create a new layer for the main rectangle
        rectangle_layer = Image.new('RGBA', result.size, (0, 0, 0, 0))
        draw = ImageDraw.Draw(rectangle_layer)
        draw.rounded_rectangle(
            (x0, y0, x1, y1),
            radius=self.corner_radius,
            fill=(*self.background_color, 255),
            outline=self.border_color[:3],
            width=self.border_width
        )
        print(y1 - y0)
        
        # Draw control buttons on the rectangle layer
        #self._draw_control_buttons(draw, x0, y0)
        # Draw horizontal line underneath the control buttons
        # reduce line opacity to 50%   
        #draw.line((x0, y0 + 130, x1, y0 + 130), fill=self.border_color, width=self.border_width)
        #draw.line((x0, y0 + 130, x1, y0 + 130), fill=(41, 117, 102), width=self.border_width)
        # draw horinzontal line at the bottom of the rectangle  in rgba

        # Open the logo images
        logo_howai = Image.open("logo_howai.png")
        logo_howai = logo_howai.resize((int(logo_howai.width * 0.5), int(logo_howai.height * 0.5)))
        logo_lang = Image.open("python2_logo.png")#f"{language}_logo.png")
        logo_lang = logo_lang.resize((int(logo_lang.width * 0.9), int(logo_lang.height * 0.9)))

        # Composite the rectangle layer
        result = Image.alpha_composite(result, rectangle_layer)
        
        # Calculate position for left-aligned code
        code_x = x0 + padding_left  # Left align with padding
        code_y = y0 + padding_top   # Top align with padding
        
        # Create a new layer for the code
        code_layer = Image.new('RGBA', result.size, (0, 0, 0, 0))
        code_layer.paste(code_image, (code_x, code_y))

        # paste logo_howai and logo_lang
        code_layer.paste(logo_howai,(0 , code_layer.height-200 ))
        code_layer.paste(logo_lang,(x0 + 60, y0+60 ))

        
        # Final composition
        result = Image.alpha_composite(result, code_layer)
        
        return result

# Example usage
#if __name__ == "__main__":
example_code = """
print("Hello, Daniel !"
def hello_world():
    Kevin = "Hello, World!"
for i in range(10):
    print(i)
"""
    
visualizer = CodeSnippetVisualizer(
    "gradient canvas.png",
    min_width=2500,
    padding_left=100-28,    # Left padding
    padding_right=100,   # Right padding
    padding_top=220,#275,     # More padding on top
    padding_bottom=100,#180,    # Less padding on bottom
    background_color=(39, 40, 34), #monokai_style
    language='python2'
)
final_image = visualizer.create_visualization(example_code)
final_image.show()
final_image.save("code_visualization_python.png")

770


In [185]:
multiline_string = """
print("Hello, World!")
"""
# get number of characters in each line
lines = multiline_string.split("\n")
for line in lines:
    print(len(line))

0
22
0


In [8]:
bg = Image.new("RGB", (800, 600), "green")
logo = Image.open("logo_howai.png")
bg.paste(logo, )
bg.show()

In [183]:
# get documentation of pygmetns monokai style
from pygments.styles import get_all_styles, get_style_by_name
style = get_style_by_name("monokai")
print(style.background_color)

#272822


# Halo effekt


# blox on blue cyan

In [148]:
# open the image
image = Image.open("brain_on_gradient_canvas.png")

# get x and y coordinates of the center
center_x = image.width // 2
center_y = image.height // 2
print(center_x, center_y)

# draw a rounded rectangle on the image
draw = ImageDraw.Draw(image)
# Assuming center_x and center_y represent the center of the rectangle
width = image.width // 2
height = image.height // 3
x0 = center_x - width // 2
y0 = center_y - height // 2
x1 = center_x + width // 2
y1 = center_y + height // 2

# Create piece of canvas to draw text on and blur
blurred = Image.new('RGBA', image.size)
draw_blurred = ImageDraw.Draw(blurred)
draw_blurred.rounded_rectangle(
    (x0, y0, x1, y1), 
    radius=20, 
    fill=(83, 234, 205, 255)  # Added 255 for full opacity
)
#draw_blurred.rounded_rectangle((x0, y0, x1, y1), radius=20, fill=(83, 234, 205))
#draw_blurred.polygon([(x0+30, y1), (x1, y0+40), (x1, y1)], fill=(83, 234, 205))#(35, 35, 35)) # draw a triangle on the image
blurred = blurred.filter(ImageFilter.GaussianBlur(50))
image.paste(blurred, (0, 0), blurred)


draw.rounded_rectangle((x0, y0, x1, y1), radius=20, fill=(35, 35, 35), outline=(83, 234, 205), width=5)

# draw a red circle on the image
draw.circle((x0+115, y0+100), 28, fill=(255, 96, 92))
draw.circle((x0+200, y0+100), 28, fill=(255, 189, 68))
draw.circle((x0+285, y0+100), 28, fill=(0, 202, 78))


# show the image
image.show()

1920 1080


# Halo effekt


In [4]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageOps

# Step 1: Create a black background image
width, height = 800, 600
background = Image.new('RGB', (width, height), 'black')

# Step 2: Draw the text or shape
text = "Halo Effect"
font_size = 100
font = ImageFont.truetype("arial.ttf", font_size)
text_image = Image.new('L', (width, height), 0)  # Create a grayscale image for the text
draw = ImageDraw.Draw(text_image)
text_width, text_height = draw.textbbox((0, 0), text, font=font)[2:]
text_x = (width - text_width) // 2
text_y = (height - text_height) // 2
draw.text((text_x, text_y), text, fill=255, font=font)  # Draw white text

# Step 3: Create a blurred version of the text for the halo
halo = text_image.filter(ImageFilter.GaussianBlur(10))  # Adjust the blur radius as needed

# Step 4: Composite the images together
halo_image = Image.new('RGB', (width, height), 'black')
halo_image.paste(ImageOps.colorize(halo, (0, 0, 0), (255, 255, 255)), (0, 0), halo)
halo_image.paste(text_image, (0, 0), text_image)

# Save or display the result
halo_image.show()